http://blog.csdn.net/wqfhenanxc/article/details/78525730

In [1]:
import numpy as np  
import pandas as pd  
import tushare as ts  
import datetime  
import time  
import os

In [2]:
import pymssql
from sqlalchemy import create_engine 
engine = create_engine("mssql+pymssql://sa:Pass0330@./invest")
cnx = engine.connect()

In [5]:
%%time
df_l = ts.get_stock_basics()

Wall time: 1.89 s


In [6]:
df_l.shape

(3505, 22)

In [9]:
df_l.head(1)

,name,industry,area,pe,outstanding,totals,totalAssets,liquidAssets,fixedAssets,reserved,...,bvps,pb,timeToMarket,undp,perundp,rev,profit,gpr,npr,holders
code,,,,,,,,,,,,,,,,,,,,,
600249,两面针,日用化工,广西,0.0,5.5,5.5,323929.22,99010.27,114079.5,85739.86,...,3.52,1.83,20040130,11563.82,0.21,-5.29,23.7,20.24,-6.53,62924.0


In [32]:
sql_str = 'SELECT Stock_ID from db_status'
df_exist = pd.read_sql(sql_str,cnx)

In [10]:
df_exist.head(1)

,Stock_ID
0,603890


In [35]:
s1 = set(df_l.index);
s2 = set(df_exist['Stock_ID'])
new = s1 - s2
len(new)

0

In [27]:
df_new = df_l.loc[new,['name','industry']]
df_new['Stock_ID'] = df_new.index

In [30]:
df_new.to_sql('db_status', cnx, if_exists='append', index=False)

In [31]:
def update_stock_list():
    df_l = ts.get_stock_basics()  # generally use 2s
    time.sleep(5)
    sql_str = 'SELECT Stock_ID from db_status'
    df_exist = pd.read_sql(sql_str,cnx)
    s1 = set(df_l.index);
    s2 = set(df_exist['Stock_ID'])
    new = s1 - s2
    df_new = df_l.loc[new,['name','industry']]
    df_new['Stock_ID'] = df_new.index
    df_new.to_sql('db_status', cnx, if_exists='append', index=False)
    return len(new)

,name,industry,Stock_ID
code,,,
603871,嘉友国际,仓储物流,603871
300739,明阳电路,元器件,300739
300738,奥飞数据,电信运营,300738
300634,彩讯股份,软件服务,300634
300504,天邑股份,通信设备,300504


In [19]:
symbol = u'600438'

str_sql =  'update db_status set Tick = 1 where Stock_ID = ' +symbol
result = cnx.execute(str_sql)

Circle = 'Tick'
str_sql = 'select Stock_ID from db_status where '+ Circle + '= 1' 
aldt = pd.read_sql(str_sql,cnx)
alL = aldt['Stock_ID'].values.tolist()
result = '600438' in alL
result

True

In [3]:
cal_dates = ts.trade_cal() #返回交易所日历，类型为DataFrame, calendarDate  isOpen

In [4]:
dateStart = '2017-01-01'
dateEnd = '2017-11-28'
date = cal_dates[(cal_dates.calendarDate>=dateStart)&(cal_dates.calendarDate < dateEnd)&(cal_dates.isOpen ==1)]
date.columns=[u'date',u'isOpen']

symbol ='600438'
str_sql = 'select date,count(1) as a from tick_'+symbol +' group by date'   
aldt = pd.read_sql(str_sql,cnx)
dt = pd.merge(date,aldt,how='left',on='date')
dt = dt[dt.a.isnull()]


In [20]:
date = '2017-11-22'
symbol = '002310'
str_date=str(date)
form_name = 'tick_' + symbol

d = ts.get_tick_data(symbol, str_date, pause = 0.1)

In [21]:
date_col = []
for i in range(len(d)):
    date_col.append(date)
d['date'] = date_col

In [22]:
type(d['type'][2])

str

In [ ]:
u_type = []
for item in d['type']:
    u_item = item.decode('utf-8')
    u_type.append(u_item)
u_type

In [21]:
# type(d['type'][2]) = unicode

unicode

In [24]:
d.to_sql(form_name,cnx, if_exists = 'append',index = False,chunksize = 500)

#根据分笔成交数据生成1分钟线  

In [25]:
type(d['date'][2])

str

In [23]:
d['time'] = d['date'] +' '+ d['time']
#d['DateTime']

In [25]:
d['time'] = pd.to_datetime(d['time'])
type(d['time'][2])
d

,time,price,change,volume,amount,type,date
0,2017-11-22 15:00:03,20.46,--,1044,2136024,买盘,2017-11-22
1,2017-11-22 14:57:00,20.46,--,3,6138,买盘,2017-11-22
2,2017-11-22 14:56:54,20.46,--,6,12276,卖盘,2017-11-22
3,2017-11-22 14:56:51,20.46,--,4,8184,买盘,2017-11-22
4,2017-11-22 14:56:48,20.46,--,35,71610,买盘,2017-11-22
5,2017-11-22 14:56:42,20.46,0.01,13,26598,买盘,2017-11-22
6,2017-11-22 14:56:39,20.45,--,10,20450,买盘,2017-11-22
7,2017-11-22 14:56:36,20.45,--,32,65440,买盘,2017-11-22
8,2017-11-22 14:56:33,20.45,--,18,36810,买盘,2017-11-22
9,2017-11-22 14:56:30,20.45,--,25,51125,买盘,2017-11-22


In [27]:
d = d.set_index('time')
# use set_index create DatetimeIndex, the col name must be 'time' or 'date'
# d.axes -- to view the info. of index
# d.info() -- to view infor DF

In [28]:
price_df = d['price'].resample('1min').ohlc()
#price_df

In [10]:
price_df=price_df.dropna() # 线性填充
#price_df

In [11]:
vols=d['volume'].resample('1min').sum() 
vols=vols.dropna()
vol_df=pd.DataFrame(vols,columns=['volume'])
amounts=d['amount'].resample('1min').sum()  
amounts=amounts.dropna()  
amount_df=pd.DataFrame(amounts,columns=['amount'])  

In [12]:
df = price_df.merge(vol_df,left_index = True,right_index=True).merge(amount_df,left_index = True, right_index=True)

In [ ]:
df

In [16]:
form_name = 'M1_' + symbol
df.to_sql(form_name,cnx, if_exists = 'append',chunksize = 500)